In [1]:
import numpy as np

In [2]:
# Sigmoid activation
def sigmoid(output):
  return 1.0 / (1.0 + np.exp(-output))

# Derivative of sigmoid function, used in backpropogation
def sigmoid_prime(output):
  return sigmoid(output) * (1 - sigmoid(output))

In [9]:
# Class for defining a layer
class Layer:
  def __init__(self, n):
    self.zs = np.zeros((n, 1)) # Weighted sum and bias
    self.activations = np.zeros((n, 1)) # Output of neuron after applying activation function
    self.n = n # Number of neurons in the current layer

  def set_weight_bias_matrix(self, prev_n): # Initialize weight and bias matrices, along with gradients
    self.weights = np.random.normal(loc = 0.0, scale = 1.0, size = (self.n, prev_n)) # Weight matrix
    self.biases = np.random.normal(loc = 0.0, scale = 1.0, size = (self.n, 1)) # Bias vector
    self.weight_gradients = np.zeros((self.n, prev_n)) # Weight gradient matrix
    self.bias_gradients = np.zeros((self.n, 1)) # Bias gradient vector

  def set_activations(self, input, input_layer = False): # Calculate activations
    n1 = len(self.activations)
    if not input_layer:
      self.zs = self.weights @ input + self.biases # Weighted sum
      self.activations = sigmoid(self.zs) # Activation

    else:
      self.activations = np.array(input).reshape((-1, 1)) # Set neuron activation to input values for input layer

  def get_activations(self): # Return activations
    return self.activations

  def get_z(self): # Return weighted sum
    return self.zs

  def get_weights(self): # Return weights
    return self.weights

  def calc_gradients(self, gradient, activations): # Gradient calculation
    gradient = np.resize(gradient, self.zs.shape) # Resize gradient matrix for matrix multiplication
    self.weight_gradients = gradient * sigmoid_prime(self.zs) * activations.T # Weight gradients
    self.bias_gradients = gradient * sigmoid_prime(self.zs) # Bias gradients
    return self.bias_gradients

  def update_params(self, learning_rate): # Update parameters
    self.weights -= learning_rate * self.weight_gradients
    self.biases -= learning_rate * self.bias_gradients

In [10]:
# Class for defining feed forward network
class Network:
  def __init__(self):
    self.network = [] # Object for holding layers
    self.lengths = [] # Object for holding lengths of all layers
    self.network_length = 0 # Network length

  # Function for adding layers
  def add(self, n):
    self.lengths.append(n)
    layer = Layer(n)
    self.network.append(layer)
    self.network_length += 1
    if self.network_length > 1:
      layer.set_weight_bias_matrix(self.lengths[-2]) # For layers after input layer, set weight and bias matrices

  # Training function
  def train(self, X, y, learning_rate):
    input_length = len(X)
    no_of_losses = 0

    for i in range(input_length):
      # Forward pass
      entry = X[i] # Input
      y_true = y[i] # Expected output
      self.network[0].set_activations(entry, input_layer = True) # Input layer
      for j in range(1, self.network_length):
        self.network[j].set_activations(self.network[j - 1].get_activations()) # Calculate weighted sum using previous layer's activations

      output = self.network[-1].get_activations() # Final output

      # For classification
      if output > 0.5:
        y_pred = 1

      else:
        y_pred = 0

      if y_true != y_pred:
        no_of_losses += 1

      loss = (y_true - output) ** 2 # Loss function MSE

      gradient = np.array([-2 * (y_true - output)]) # Gradient of loss

      # Backward pass
      for j in range(self.network_length - 1, 0, -1):
        gradient = self.network[j].calc_gradients(gradient, self.network[j - 1].get_activations()) # Calculate weight and bias gradients
        if j > 1:
          weights = self.network[j].get_weights()
          gradient = weights.T @ gradient # Calculate gradient to be backpropogated

      for k in range(self.network_length - 1, 0, -1): # Final forward pass to update all layer parameters
        self.network[k].update_params(learning_rate)

    print("Accuracy: ", ((input_length - no_of_losses) / input_length) * 100) # Final training accuracy

  # Testing function
  def test(self, X, y):
    input_length = len(X)
    no_of_losses = 0
    for i in range(input_length):
      entry = X[i]
      y_true = y[i]
      self.network[0].set_activations(entry, input_layer = True)
      for j in range(1, self.network_length):
        self.network[j].set_activations(self.network[j - 1].get_activations())

      output = self.network[-1].get_activations()

      if output > 0.5:
        y_pred = 1

      else:
        y_pred = 0

      if y_true != y_pred:
        no_of_losses += 1

    print("Accuracy: ", ((input_length - no_of_losses) / input_length) * 100)

### Sample code cell for training and testing

In [ ]:
# import pandas as pd

# df_train = pd.read_csv("/content/train_sub.csv", header = None)
# df_test = pd.read_csv("/content/test.csv", header = None)

# df_train.drop(0, axis = 1, inplace = True)
# df_train.columns = range(df_train.shape[1])
# input_length = len(df_train.columns) - 1

# model = Network()

# model.add(input_length)
# model.add(2)
# model.add(3)
# model.add(2)
# model.add(3)
# model.add(1)

# print("Training:")
# for _ in range(10):
#   print("Iteration ", _+1)
#   temp = df_train.sample(frac = 1, ignore_index = True).values

#   X_train = temp[:, 1:]
#   X_train = X_train / 255
#   train_labels = temp[:, 0]

# print("Test:")
# test_temp = df_test.values

# X_test = test_temp[:, 1:]
# X_test = X_test / 255
# test_labels = test_temp[:, 0]

# model.test(X_test, test_labels)